# CPU and Memory Profiling

Profiling CPU and memory with Python.

## Timing code

We can use `time.time()` to time code.

In [1]:
import time

def is_prime_fn1(x):
    for i in range(2, x-1):
        if x % i == 0:
            return False
    return True

def is_prime_fn2(x):
    for i in range(2, int(x**.5)+1):
        if x % i == 0:
            return False
    return True

is_prime_lambda = lambda x: all(x % i != 0 for i in range(2, int(x**.5)+1))

def get_primes(y, func):
    _primes = []
    for val in range(y):
        if func(val):
            _primes.append(val)
    return _primes

start_time = time.time()
print(get_primes(1000, is_prime_fn1))
print(get_primes(1000, is_prime_fn2))
print(get_primes(1000, is_prime_lambda))
end_time = time.time()
print(f"elapsed time = {end_time - start_time} seconds")

[0, 1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
[0, 1, 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179,

We can use `time.perf_counter()` for system wide elapsed time...

In [2]:
start_time = time.perf_counter()            # time.clock() deprecated in Python 3.8
get_primes(10000, is_prime_fn1)
print(f"is_prime_fn1 elapsed time = {time.perf_counter()- start_time} seconds")

start_time = time.perf_counter()
get_primes(10000, is_prime_fn2)
print(f"is_prime_fn2 elapsed time = {time.perf_counter()- start_time} seconds")

start_time = time.perf_counter()
get_primes(10000, is_prime_lambda)
print(f"is_prime_lambda elapsed time = {time.perf_counter()- start_time} seconds")

is_prime_fn1 elapsed time = 1.8364242000097875 seconds
is_prime_fn2 elapsed time = 0.10650799999712035 seconds
is_prime_lambda elapsed time = 0.2512453999952413 seconds


We can use `time.process_time()` if only want time for this process, excluding any sleeps...

In [3]:
start_time = time.process_time()
get_primes(10000, is_prime_lambda)
print(f"is_prime_lambda elapsed time = {time.process_time()- start_time} seconds")

is_prime_lambda elapsed time = 0.09375 seconds


We can use `time.process_time_ns()` for nanoseconds counting...

In [4]:
start_time = time.perf_counter_ns()
print("hello!")
print(f"print elapsed time = {time.perf_counter_ns()- start_time}ns")

hello!
print elapsed time = 1338100ns


We can use `timeit.timeit()` if you prefer...

In [5]:
import timeit

print(timeit.timeit("get_primes(10000, is_prime_fn1)", globals=globals(), number=5))
print(timeit.timeit("get_primes(10000, is_prime_fn2)", globals=globals(), number=5))
print(timeit.timeit("get_primes(10000, is_prime_lambda)", globals=globals(), number=5))

4.676494700004696
0.1115523999906145
0.538807100005215


We can use a decorator if we so wish...

In [6]:

import statistics       # in standard library since Python 3.4/PEP450

def time_primes(number_tests=1):
    def my_time_prime_decorator(func):
        def time_prime_execution(*args, **kwargs):
            _tests = []
            for t in range(1, number_tests):
                _start_time = time.process_time()
                func(*args, **kwargs)
                _end_time = time.process_time()
                _tests.append(_end_time - _start_time)
            print("number of tests executed =", number_tests)
            print("mean execution time =", statistics.mean(_tests))
            print("standard deviation execution time =", statistics.stdev(_tests))
        return time_prime_execution
    return my_time_prime_decorator

@time_primes(number_tests=10)
def get_primes(y, func):
    _primes = []
    for val in range(y):
        if func(val):
            _primes.append(val)
    return _primes

get_primes(10000, is_prime_fn1)
get_primes(10000, is_prime_fn2)
get_primes(10000, is_prime_lambda)

number of tests executed = 10
mean execution time = 0.7170138888888888
standard deviation execution time = 0.0824744146985058
number of tests executed = 10
mean execution time = 0.020833333333333332
standard deviation execution time = 0.0078125
number of tests executed = 10
mean execution time = 0.03819444444444445
standard deviation execution time = 0.01135129933213717


For large lists of data it is often much faster to create a dictionary than perform linear search...

In [7]:
my_big_data_list = [(7263, 'bob'), (221333, 'sally'), (212892, 'simon')]
for x in my_big_data_list:
    if x[0] == 221333:
        print("found them!")
my_big_data_list_dict = {x[0]: x for x in my_big_data_list}         # use a dictionary comprehension
print(my_big_data_list_dict)
print(my_big_data_list_dict[221333])

found them!
{7263: (7263, 'bob'), 221333: (221333, 'sally'), 212892: (212892, 'simon')}
(221333, 'sally')


## Memory usage

In [8]:
from pympler import asizeof
import sys

my_number_list = [1, 2, 3, 4]
print("some objects have the __sizeof__ attribute")
print("my_list =", my_number_list)
print(dir(my_number_list))
print("type(my_list.__sizeof__) =", type(my_number_list.__sizeof__))
print("len(my_list) =", len(my_number_list))                               # just number of elements
print("my_list.__sizeof__() =", my_number_list.__sizeof__())               # raw object size
print("sys.getsizeof(1) =", sys.getsizeof(1))
print("sys.getsizeof(my_number_list) =", sys.getsizeof(my_number_list))    # __sizeof__ + garbage collector overhead
print("len(my_number_list) * (1).__sizeof__() =", len(my_number_list) * (1).__sizeof__())
print("total size of elements greater than list object size!")

my_string_list = ['hello world', 'my name is Bob', 'quick brown fox', 'phillip the cat']
print("my_string_list =", my_string_list)
print("len(my_string_list) =", len(my_string_list))                         # just number of elements
print("my_string_list.__sizeof__() =", my_string_list.__sizeof__())         # raw object size
print("sys.getsizeof(my_string_list) =", sys.getsizeof(my_string_list))     # __sizeof__ + garbage collector overhead
print("string list the same size as integer list")

print("sys.getsizeof(my_number_list) + sys.getsizeof(1) + ... =",
      sys.getsizeof(my_number_list) +
      sys.getsizeof(1) +
      sys.getsizeof(2) +
      sys.getsizeof(3) +
      sys.getsizeof(4))
print("sys.getsizeof(my_string_list) + sys.getsizeof('hello world') + ... =",
      sys.getsizeof(my_string_list) +
      sys.getsizeof('hello world') +
      sys.getsizeof('my name is Bob') +
      sys.getsizeof('quick brown fox') +
      sys.getsizeof('phillip the cat'))
print("asizeof.asizeof(my_number_list) =", asizeof.asizeof(my_number_list))
print("asizeof.asizeof(my_string_list) =", asizeof.asizeof(my_string_list))

some objects have the __sizeof__ attribute
my_list = [1, 2, 3, 4]
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']
type(my_list.__sizeof__) = <class 'builtin_function_or_method'>
len(my_list) = 4
my_list.__sizeof__() = 72
sys.getsizeof(1) = 28
sys.getsizeof(my_number_list) = 88
len(my_number_list) * (1).__sizeof__() = 112
total size of elements greater than list object size!
my_string_list = ['hello world', 'my name is Bob', 'quick brown fox', 'phillip the cat'

Using tracemalloc...

In [9]:
import tracemalloc

tracemalloc.start()
trace_malloc_vector = [z for z in range(1000)]
memory_snapshot = tracemalloc.take_snapshot()
stats = memory_snapshot.statistics('lineno')
for stat in stats[:10]:
    print(stat)

C:\Users\Mike\AppData\Local\Temp\ipykernel_1948\624066326.py:4: size=28.9 KiB, count=744, average=40 B
C:\Users\Mike\AppData\Local\Temp\ipykernel_1948\624066326.py:5: size=416 B, count=1, average=416 B
c:\python310\lib\codeop.py:118: size=256 B, count=3, average=85 B
C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3373: size=112 B, count=1, average=112 B
C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3363: size=96 B, count=3, average=32 B
C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3422: size=64 B, count=1, average=64 B
C:\Users\Mike\AppData\Roaming\Python\Python310\site-packages\IPython\core\compilerop.py:177: size=28 B, count=1, average=28 B


Using guppy...

In [10]:
from guppy import hpy

h = hpy()
h.setrelheap()
my_number_heaped_list = ['red', 'brown', 'green', 'blue']
print(my_number_heaped_list)
result = h.heap()
print(result)

['red', 'brown', 'green', 'blue']
Partition of a set of 23 objects. Total size = 2660 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      3  13     1320  50      1320  50 types.FrameType
     1      3  13      208   8      1528  57 list
     2      3  13      192   7      1720  65 types.MethodType
     3      1   4      176   7      1896  71 types.CodeType
     4      3  13      160   6      2056  77 tuple
     5      1   4      112   4      2168  82 asyncio.events.TimerHandle
     6      1   4      104   4      2272  85 dict of ast.Module
     7      2   9       80   3      2352  88 bytes
     8      1   4       80   3      2432  91 functools.partial
     9      1   4       64   2      2496  94 _contextvars.Context
<4 more rows. Type e.g. '_.more' to view.>


Using memory_profiler...

In [11]:
from memory_profiler import profile

@profile
def my_func():
    a = [1] * (10 ** 6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a

my_func()

ERROR: Could not find file C:\Users\Mike\AppData\Local\Temp\ipykernel_1948\3959980297.py


[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


Using objgraph...

In [12]:
import objgraph

objgraph.show_most_common_types()
objgraph.show_growth(limit=3)
my_new_number_list = [1, 2, 3, 4]
my_new_string_list = ['hello world', 'my name is Bob', 'quick brown fox', 'phillip the cat']
objgraph.show_growth()

dict                       27137
function                   22445
list                       15708
tuple                      14468
Operator                   7926
ReferenceType              6835
DFAPlan                    5418
Name                       5002
PythonNode                 4288
builtin_function_or_method 3813
dict        27135    +27135
function    22445    +22445
list        15708    +15708
list           15709        +1
Interactive        2        +1
